In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install bitsandbytes transformers datasets accelerate peft torch torchvision torchaudio datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.6 MB/s eta 0:00:00:00:0100:01


https://github.com/lisaalaz/satbot/blob/master/model/EmpatheticPersonas.csv
last step training on empathetic dialogue

!git clone https://github.com/thu-coai/Emotional-Support-Conversation.git

In [1]:
from datasets import load_dataset

# Load dataset
dataset_name = "Amod/mental_health_counseling_conversations"
mental_health_dataset = load_dataset(dataset_name)

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
mental_health_dataset["train"][0]

In [5]:
# Split the mental health dataset
train_test_split = mental_health_dataset["train"].train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]


print("Dataset formatted for QLoRA fine-tuning.")

Dataset formatted for QLoRA fine-tuning.


In [ ]:
len(train_dataset)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model_path = '/kaggle/input/phi2_train_on_1_epoch_3000/other/default/2/fine_tuned_phi2_multiconv'
model_name = "microsoft/Phi-3-mini-4k-instruct"


# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16 
)

print("✅ Phi3.5 LLM Loaded with LoRA and 4-bit Precision!")

In [ ]:
from peft import LoraConfig, get_peft_model
# Apply LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    r=4,  # Low-rank adaptation size
    lora_alpha=32,
    target_modules=["qkv_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
print("EOS Token:", tokenizer.eos_token)  # Should print "<|endoftext|>"
print("EOS Token ID:", tokenizer.eos_token_id)  # Should print an integer


In [7]:
max_length = model.config.max_position_embeddings

In [ ]:
import json
from datasets import Dataset

def format_for_finetuning(entry, tokenizer, max_length):
    context = entry["Context"]
    response = entry["Response"]

    # Combine system, user context, and the response.
    system_message = "<|system|>\nYou are a friendly therapist, respond to the users' problems in a helpful manner.<|end|>\n"
    user_message = f"<|user|>\n{context}<|end|>\n"
    response_message = f"<|assistant|>\n{response}<|end|>\n<|endoftext|>"

    # Combine all parts
    combined_input = system_message + user_message + response_message

    # Tokenize the combined sequence
    tokenized_input = tokenizer(combined_input, truncation=True, max_length=max_length, padding="max_length")

    # Tokenize each part to get correct indices
    system_tokens = tokenizer.encode(system_message)
    user_tokens = tokenizer.encode(user_message)
    response_tokens = tokenizer.encode(response_message)

    # Create labels: -100 for the system and user parts, and valid labels for the response
    labels = [-100] * len(system_tokens) + [-100] * len(user_tokens) + response_tokens
    # Truncate labels to max_length to avoid exceeding the model's limit
    labels = labels[:max_length]
    
    padding_length = len(tokenized_input["input_ids"]) - len(labels)
    if padding_length > 0:
        # Pad labels on the left (to match padding of the tokenized input)
        labels = [-100] * padding_length + labels

    tokenized_input["labels"] = labels

    return tokenized_input

def process_and_format(dataset, tokenizer, max_length):
    # Use map to process data in a memory-efficient way
    return dataset.map(lambda entry: format_for_finetuning(entry, tokenizer, max_length), batched=False)

# Example usage
dataset_train_formatted = process_and_format(train_dataset, tokenizer, max_length=max_length)
dataset_test_formatted = process_and_format(test_dataset, tokenizer, max_length=max_length)


In [ ]:
print(dataset_train_formatted[0])
print(len(dataset_train_formatted[0]["input_ids"]))
print(len(dataset_train_formatted[0]["attention_mask"]))
print(len(dataset_train_formatted[0]["labels"]))

Load model

In [ ]:
ls -lh /kaggle/working

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments

print("✅ WandB Disabled!")

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    evaluation_strategy="no",
    learning_rate=3e-4,  # Lower learning rate for LoRA fine-tuning
    per_device_train_batch_size=1,  # Reduce batch size for memory efficiency
    gradient_accumulation_steps=4,  # Simulate larger batch size
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,  # Mixed precision training
)

In [ ]:
from transformers import TrainingArguments, Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_formatted,
    # eval_dataset=dataset_test_formatted,
)
print("🚀 Trainer Initialized!")

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./fine_tuned_phi35_mini")
tokenizer.save_pretrained("./fine_tuned_phi35_mini")
print("✅ Fine-Tuned Model Saved Successfully!")

In [ ]:
import shutil
shutil.move("./fine_tuned_phi35_mini", "/kaggle/working/fine_tuned_phi35_mini")

In [ ]:
import torch
torch.cuda.empty_cache()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_path = '/kaggle/input/microsoft-phi-2-trained/pytorch/default/1/fine_tuned_phi2_train2'
model_path = "/kaggle/input/phi3_train/pytorch/default/1/results (5)/fine_tuned_phi35_mini"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

print("✅ Fine-Tuned Model Loaded Successfully!")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

✅ Fine-Tuned Model Loaded Successfully!


In [27]:
import json
from datasets import Dataset

def format_conversation_test(data, tokenizer, max_length=1221):
    
    context = data["Context"]
    response = data["Response"]
    
    # Combine system, user context, and the response.
    system_message = "<|system|>\nYou are a friendly therapist, respond to the users' problems in a helpful manner.<|end|>\n"
    user_message = f"<|user|>\n{context}<|end|>\n"
    response_message = "<|assistant|>\n"

    # Combine all parts for tokenization
    combined_input = system_message + user_message + response_message

    # Tokenize the combined sequence for test data
    tokenized_input = tokenizer(combined_input, truncation=True, max_length=max_length)
    
    # The test data typically won't have labels, so no need to add them
    tokenized_input["labels"] = None
    
    return tokenized_input

def process_and_format(dataset, tokenizer, max_length):
    # Use map to process data in a memory-efficient way
    return dataset.map(lambda entry: format_conversation_test(entry, tokenizer, max_length), batched=False)

dataset_test_formatted = process_and_format(test_dataset, tokenizer, max_length=max_length)

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [29]:
import torch

idx = [1,3,5]
for i in range(len(idx)):
    input_ids = torch.tensor([dataset_test_formatted[i]["input_ids"]]).to(model.device)

    # Optional: also pass attention_mask if needed
    attention_mask = torch.tensor([dataset_test_formatted[i]["attention_mask"]]).to(model.device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True))

You are a friendly therapist, respond to the users' problems in a helpful manner. How does a counselor decide when to end counseling sessions or to terminate working with a client? Counseling is a process, not a product.  The client and the counselor are
You are a friendly therapist, respond to the users' problems in a helpful manner. My boyfriend and I have been arguing every night about the same thing. He also tells me that if I go visit my mother out of state he will be gone when I get back. He and my mother do not get along. What should I do? It sounds like you are feeling very frustrated and upset about your relationship. It is important
You are a friendly therapist, respond to the users' problems in a helpful manner. I have high functioning autism and I have been on a lot of dating sites like meet me, match, and zoosk. I haven't had any luck on any of the dating sites I have been on.

 I really want a boyfriend but I don't know what I should do. I just want to be in a relationshi

Original model

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/kaggle/working/fine_tuned_phi35_mini"
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

print("✅ Fine-Tuned Model Loaded Successfully!")


In [21]:
dataset_test_formatted = dataset_test_formatted.remove_columns(["Context", "Response"])

In [ ]:
idx = [1,3,5]
for i in range(len(idx)):
    input_ids = torch.tensor([dataset_test_formatted[i]["input_ids"]]).to(model.device)

    # Optional: also pass attention_mask if needed
    attention_mask = torch.tensor([dataset_test_formatted[i]["attention_mask"]]).to(model.device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True))